# Python vs MySQL

Để truy cập MySQL từ Python cần phải có một driver

Cài đặt driver

`pip install PyMySQL`

In [1]:
# Kết nối đến MySQL
from pymysql import connect, cursors

config = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "demo",
}

conn = connect(**config)
conn.close()

In [2]:
# Xử lý exception, tự đóng kết nối
from pymysql import connect, cursors, Error

config = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "demo",
    "cursorclass": cursors.DictCursor
}

try:
    with connect(**config) as conn:
        print("Connected to database:", conn.db.decode("UTF-8"))
except Error as e:
    print(e)

Connected to database: demo


In [4]:
# Truy vấn dữ liệu
def get_all_students():
    sql = '''SELECT * FROM students'''

    try:
        with connect(**config) as conn:
            # Cursor Object đại diện cho con trỏ trong CSDL
            # Có nhiệm vụ tìm nạp dữ liệu, thực thi câu lệnh
            with conn.cursor() as cur:
                # Thực thi câu lệnh
                cur.execute(sql)
                
                # Các hàm lấy dữ liệu:
                # - fetchone() - Đọc 1 record
                # - fetchall() - Đọc toàn bộ records
                # - fetchmany(size) - Đọc một số records
                return cur.fetchall()
    except Error as e:
        print(e)

def get_student_by_id(id):
    # SQL Injection
    # Với các truy vấn cần chèn giá trị
    # Sử dụng mẫu pyformat
    sql = '''SELECT * FROM students WHERE id = %(id)s'''

    try:
        with connect(**config) as conn:
            with conn.cursor() as cur:
                # Truyền args cho params
                cur.execute(sql, {'id': id})
                return cur.fetchone()
    except Error as e:
        print(e)

student = get_student_by_id(102)

if student is not None:
    id, name, dob, email = student.values()
    print(id, name, dob, email)

102 Barrr 1992-05-24 ba@techmaster.vn


In [5]:
# Thêm dữ liệu
def add_student(student):
    sql = '''
    INSERT INTO students (name, dob, email)
    VALUES (%(name)s, %(dob)s, %(email)s)
    '''

    try:
        with connect(**config) as conn:
            with conn.cursor() as cur:
                cur.execute(sql, student)
                conn.commit()
                return cur.lastrowid
    except Error as e:
        print(e)

new_student = {
    "name": input("Name: "),
    "dob": input("DOB: "),
    "email": input("Email: ")
}

print(add_student(new_student))

102


In [6]:
# Cập nhật dữ liệu
def update_student_info(student):
    sql = '''
    UPDATE students
    SET name = %(name)s,
        dob = %(dob)s,
        email = %(email)s
    WHERE id = %(id)s
    '''

    try:
        with connect(**config) as conn:
            with conn.cursor() as cur:
                cur.execute(sql, student)

                if cur.rowcount:
                    conn.commit()
                    return True
                else:
                    return False
    except Error as e:
        print(e)

student = get_student_by_id(102)

if student is not None:
    print("Cập nhật thông tin học viên!")
    print("Nhấn Enter để bỏ qua, giữ nguyên giá trị hiện tại")

    for key in student:
        if key == "id": continue

        label = f"{key[0].upper() + key[1:].lower()}: ({student[key]})"
        student[key] = input(label) or student[key]
    
    print(update_student_info(student))

Cập nhật thông tin học viên!
Nhấn Enter để bỏ qua, giữ nguyên giá trị hiện tại
True


In [86]:
# Xóa dữ liệu
def delete_student_by_id(id):
    sql = '''DELETE FROM students WHERE id = %s'''

    try:
        with connect(**config) as conn:
            with conn.cursor() as cur:
                cur.execute(sql, id)

                if cur.rowcount:
                    conn.commit()
                    return True
                else:
                    return False
    except Error as e:
        print(e)

print(delete_student_by_id(101))

True
